## Predict Batch Perfromance
### Grey Atom Hackthon 2 : Dec 17 to Jan 2nd
#### Version 0 - Collapsing/Merging Sensor data

Here the attempt is to combine the sensor data for each sensor, as there is high collinnearity between the readings for each sensor. By engineering the data into one or two columns we will be able to (a) reduce the number of features; and (b) remove multi-collinearity


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [3]:
# load the dataset
pbp_org = pd.read_csv('Train.csv')


In [4]:
pbp_org.shape

(34226, 16)

for each sensor
- extact the seven reading
- calculate the mean and std dev
- determine the % deviation
- save in a df


In [176]:
%%time

# Calculating the deviation (std/mean) for each sensor
# using direct mean and std  method

sensor_var = pbp_org.Id                                                                     # start with the batch ID
print('Calculating Spread for Sensor : ', end = '')

for sensor in range (0,55) :                                                                # for each sensor
    colname = []
    print ('--', sensor, end = '')
    for reading in range (0,7) :                                                            # get the number of reading
        colname += ['x' + str(sensor) + '_t' + str(reading)]                                # get column names
        sensor_data = pbp_org[colname]                                                      # Extract data col for one sensor
        sensor_desc = np.round(sensor_data.T.std() / sensor_data.T.mean() * 100, 3)         # find spread% (std to mean) for each batch
        
    sensor_var = pd.concat([sensor_var, sensor_desc], axis = 1)                             # build for each sensor
    sensor_var.rename (columns = {0: 'x'+str(sensor)+'_spread'}, inplace = True)
    
sensor_var = pd.concat([sensor_var, pbp_org.y2], axis = 1)                                  # Append the batch result col i.e. y2


print('\n\nAdded', sensor_var.shape[1] - pbp_org.shape[1], 'more MEAN columns')

Calculating Spread for Sensor : -- 0-- 1-- 2-- 3-- 4-- 5-- 6-- 7-- 8-- 9-- 10-- 11-- 12-- 13-- 14-- 15-- 16-- 17-- 18-- 19-- 20-- 21-- 22-- 23-- 24-- 25-- 26-- 27-- 28-- 29-- 30-- 31-- 32-- 33-- 34-- 35-- 36-- 37-- 38-- 39-- 40-- 41-- 42-- 43-- 44-- 45-- 46-- 47-- 48-- 49-- 50-- 51-- 52-- 53-- 54

Added -330 more MEAN columns
Wall time: 1.46 s


In [177]:
sensor_var.iloc[:, 0:10]

,Id,x0_spread,x1_spread,x2_spread,x3_spread,x4_spread,x5_spread,x6_spread,x7_spread,x8_spread
0,706,0.980,20.964,0.0,11.732,1.123,0.438,0.336,0.288,0.000
1,239,1.081,4.019,0.0,0.405,0.000,1.782,0.000,0.000,0.000
2,613,0.751,28.181,0.0,12.916,0.000,12.506,0.149,0.055,0.000
3,588,1.018,31.037,0.0,0.770,0.000,0.660,0.149,0.055,0.000
4,780,1.039,31.087,0.0,27.669,0.555,0.879,1.132,2.853,0.000
...,...,...,...,...,...,...,...,...,...,...
789,835,0.946,29.064,0.0,14.356,2.306,1.634,0.530,0.505,0.000
790,192,1.524,43.704,0.0,1.968,2.306,0.770,2.243,1.098,0.843
791,629,1.301,20.865,0.0,13.227,0.000,0.847,0.149,0.055,0.000
792,559,0.935,14.085,0.0,35.120,0.000,0.427,0.000,0.000,0.000


In [179]:
show(sensor_var)

ANALYSIS: 
The above df 'sensor_var' show the spread among the 't0' to 't6' readings for each sensor.
The ASSUMPTION is that for a normal process, the reading should not vary by much, and hence the spread (here std-dev / mean) should not be wide. 

For abmost 20 sensors, this variation is under 5%, and for the others it is much higher.

Possible INTUTION is :
- The sensors where the reading is in the narrow range, the readings columns would be collinear, and hence ONLY one of the readings column should be used. Question is which one?
- The sendors where the the reading is in a wide range, there is little co-realation among the readings, and hence all the columns can be used.

reading the sensor spread table

- 1,3,9, 11, 15, 19, 24, 26, 28, 29, 30  : Have high spreads
- 25 : blanks
- 28 : variation of 175% +
- 37, 38 : high variation of 250% + AND many BLANKS 
- 2, 8 :  all zero



#  
#  
#  
### AVOID THE BELOW CODE . FOR TESTING AND VALIDATION PURPOSE ONLY

In [166]:
%%time

# Calculating the deviation (std/mean) for each sensor
# using .describe() method

sensor_var = pbp_org.Id
print('Calculating Mean for Sensor : ', end = '')

for sensor in range (0,55) :
    colname = []
    print ('--', sensor, end = '')
    for reading in range (0,7) :
        colname += ['x' + str(sensor) + '_t' + str(reading)]   # get column names
        sensor_data = pbp_org[colname]
        sensor_desc = sensor_data.T.describe().T               # VERY CPU RESOURCE INTENSIVE : AVOID
        sensor_data = pd.concat([sensor_data, sensor_desc['std'] / sensor_desc['mean']],axis = 1)
        sensor_data.rename (columns = {0: 'x'+str(sensor)+'_mean'}, inplace = True)
        
    sensor_var = pd.concat([sensor_var, sensor_data], axis = 1)
    
sensor_var = pd.concat([sensor_var, pbp_org.y2], axis = 1)

print('\n\nAdded', sensor_var.shape[1] - pbp_org.shape[1], 'more MEAN columns')


# NOTE :
# this is a VERY RESOURCE INTENSIVE code. This is due to this stmt : sensor_data.T.describe().T

Calculating Mean for Sensor : -- 0-- 1-- 2-- 3-- 4-- 5-- 6-- 7-- 8-- 9-- 10-- 11-- 12-- 13-- 14-- 15-- 16-- 17-- 18-- 19-- 20-- 21-- 22-- 23-- 24-- 25-- 26-- 27-- 28-- 29-- 30-- 31-- 32-- 33-- 34-- 35-- 36-- 37-- 38-- 39-- 40-- 41-- 42-- 43-- 44-- 45-- 46-- 47-- 48-- 49-- 50-- 51-- 52-- 53-- 54

Added 55 more MEAN columns
Wall time: 5min 58s
